In [ ]:
import csv
import pandas as pd
import sys

**Negative and Positive Cohort**

Psotive and Negative cohort. 1 to 1 ratio (50% positive and 50% negative)

Basically, create a new file that contains patients who were diagnosed with diabetes and other specific 
diseases listed in diseases list.

@Input file: indexed data excludes procedures. All patients in this file are diabetics <br/>
@Output file:  a new file that contains patients who developed certain disease after 4 visits

In addition, this file also keeps track of patients IDS who are positive, such that we can take
negative patients by making sure the ID is not in the list of patients who tested positive
<hr/>
First, create two seperate files for positive and negative, then, take all positive patients and the same number from the negative patients

In [ ]:
diseases = ["72"]
diseases_count = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
patient_ids_with_positive_cohort = []
output_file = 'HCUP_indexed_positive_d_311.csv'

csv_positive_cohort = open(output_file, mode='w')

file = 'HCUP_indexed_ex_procedures.csv'
with open(file) as csv_file:
    for line in csv_file:    
        row = line.split(',')

        for x in range(0, len(row)):
            
            if row[x] == '3':
                i = x
                data = row[i:] 
                # print(data)
                c = 0
                for i in range(0, len(data)):
                    if '#' in data[i]:
                        c+=1
                    if c == 2: # 2 visits
                        data_2 = data[:i]
                        data_3 = data[i:]
                        for j in range(len(diseases)):
                            for x in data_3:
                                
                                if '#' in x: x = x.split('#')[0]
                                
                                if diseases[j] == x:
                                    
                                    flag = False
                                    for y in data_2:
                                        if diseases[j] == y:
                                            flag = True
                                            break
                                    if not flag:
                                        diseases_count[j] += 1
                                        csv_positive_cohort.write(line) 
                                        patient_id = line.split('|')[0]
                                        patient_ids_with_positive_cohort.append(patient_id) # to keep track
                                        break        
                                    
                        break
                break

        
csv_positive_cohort.close()
print(diseases_count)

Now, we have a dataset that contains all positive patients. 
Lets create a file with patients/diabetics who do not have specific disease (negative/0)

In [ ]:
out_file = 'HCUP_indexed_negative_d_311.csv'
negative_cohort_file = open(out_file, mode='w')

file = 'HCUP_indexed_ex_procedures.csv'

with open(file, mode='r') as input_file:
    for row in input_file:
        patient_id = row.split('|')[0]
        
        # if patient id is not in the list of positive, then negative
        if patient_id not in patient_ids_with_positive_cohort:
            visits = row.replace('\n','').split('|')[2].split('#')
            if len(visits) >= 3:
                negative_cohort_file.write(row)

negative_cohort_file.close()
print("done")

Test and count how many negative and positive patients there are 
take all positive patients and same number of patients from negative

In [ ]:
# Test data-set
file = 'HCUP_indexed_positive_d_311.csv'
counter = 0
with open(file) as test_data:
    for row in test_data:
        counter+=1
print(counter)

# Test data-set
file = 'HCUP_indexed_negative_d_311.csv'
counter = 0
with open(file) as test_data:
    for row in test_data:
        #print(row)
        counter+=1
print(counter)

Now, we have two different datasets. Positive and negative patients

Create a new file that contains 1 to 1 ratio - 50/50 balanced data

In [ ]:
# 1 to 1 ratio - 50/50 - balanced data
file = 'HCUP_D_311_1_to_1.csv'
new_csv = open(file, mode='a')
input_file = 'HCUP_indexed_positive_d_311.csv'

counter = 0
with open(input_file, mode='r') as data:
    for row in data:
        counter += 1
        new_csv.write(row)
new_csv.close()
print('rows written', counter)
n_pos = counter
# first 21631 are positive

In [ ]:
print(n_pos)
# 1 to 1 ratio - 50/50 - balanced data
file = 'HCUP_D_311_1_to_1.csv'
new_csv = open(file, mode='a')
input_file = 'HCUP_indexed_negative_d_311.csv'

counter = 0
with open(input_file, mode='r') as data:
    for row in data:
        if counter == n_pos: break
        counter += 1
        new_csv.write(row)
new_csv.close()
# second 21631 are negative
print('rows written', counter)

Test and count total number - should be 21631*2 = 43262

<hr/>
<hr/>
<hr/>
<hr/>
<hr/>

Load med codes

In [ ]:
codes_d = {}
file = 'disease_codes.csv'
with open(file) as input_data:
    for code in input_data:
        index = code.split(',')[0][2:-1]
        code = code.split(',')[1][1:-2]
        
        codes_d[code] = index

In [ ]:
# lets take a look into the dict
print(len(codes_d))
for k, v in codes_d.items():
    print(k, '-->', v)


In [ ]:
# count how many times a code occur per patient
file = 'HCUP_D_311_1_to_1.csv'
counter = 0

code_count = {}

patient_index = 1
with open(file) as test_data:
    for row in test_data:
        visits = row.replace('\n','').split('|')[2].split('#')
        for k in range(0,len(visits)):
            visit_codes = visits[k].split(',')[1:]
            # print(visit_codes)
            for v in visit_codes:
                code_count[v] = code_count.get(v, 0) + 1

In [ ]:
# Delete those medical codes from dataset 
keys_to_delete = []
counter = 0
for k, v in code_count.items():
     if v < 50 or v > 708290:
        counter += 1
        keys_to_delete.append(k)
        
print(counter)
# delete those codes from data-set 

In [ ]:
for i in keys_to_delete:
    print(i)
    if i == '72':
        print('-------------------------')
        print(i)
        break

In [ ]:
# new medical codes is a dict that exludes codes that we wanted to delete
counter = 0
new_med_codes = {}
for k, v in codes_d.items():
    if k in keys_to_delete:
        continue
    new_med_codes[k] = v

In [ ]:
# delete procedures from new_med_codes
med_codes_without_procedures = {}
for k, v in new_med_codes.items():
    if v.startswith("P_"):
        continue
    med_codes_without_procedures[k] = v

In [ ]:
# Create a new medical codes file - Code and its index
import operator

codes_file = open('disease_codes_v2.csv', 'w')

for k, v in med_codes_without_procedures.items():
    codes_file.write('(\'' + v + '\', ' + k + ")\n")
codes_file.close() 

In [ ]:
# create a new file that excludes medical codes to delete
new_file = 'HCUP_D_311_1_to_1_v2.csv'

new_csv = open(new_file, mode='w')

file = 'HCUP_D_311_1_to_1.csv'

with open(file) as input_file:
    for line in input_file:
        front = line.replace('\n', '').split('|')[:2]
        visits = line.replace('\n','').split('|')[2].split('#')
        new_visits = []
        
        for visit in visits:
            link_and_age = visit.split(',')[:2]
            med_codes = visit.split(',')[2:]
            new_codes = []
            
            for med_code in med_codes:
                # med_code is not '-99' and
                 if med_code in med_codes_without_procedures:
                    new_codes.append(str(med_code))
            new_visits.append(','.join(link_and_age + new_codes))
        new_csv.write('|'.join(front) + '|' + '#'.join(new_visits) + '\n')
new_csv.close()

<hr/>
<hr/>
<hr/>
<hr/>
<hr/>

**MAKE AMENDMENT**

For those patients who were tested positive, remove visits after that contain the certain complication we are looking for and after
and for those who were tested negative, leave the length as it is

In [ ]:
# Cut positive patients such that we only have the row from 0 until we find the medical code of the required complication 
import sys
file = 'HCUP_D_311_1_to_1_v2.csv'
out_file = 'HCUP_D_311_1_to_1_v3.csv'
new_csv = open(out_file, mode = 'a')
counter = 0

with open(file, mode='r') as csv_file:
    for line in csv_file:
        if counter < n_pos: # positive cohorts 
            counter += 1
            front = line.replace('\n', '').split('|')[:2]
            visits = line.replace('\n','').split('|')[2].split('#')

            index = 0
            for v in visits:
                index+=1
                if '72' in v[2:]:
                    new_csv.write('|'.join(front) + '|' + '#'.join(visits[0:index-1]) + '\n')
                    break
        else:
            new_csv.write(line) 
new_csv.close()

In [ ]:
# Checks avg number of visits per patients for positive and negative class to make sure we are not biased

file = 'HCUP_D_311_1_to_1_v3.csv'

avg_len_pos = 0
avg_len_neg = 0

counter = 0
with open(file, mode='r') as csv_file:
    for line in csv_file:
        if counter < n_pos:
            avg_len_pos += line.count("#")
        else:
            avg_len_neg += line.count("#")
        counter += 1
print(avg_len_pos / n_pos)
print(avg_len_neg / n_pos)
        

<hr/>
<hr/>
<hr/>
<hr/>
<hr/>

In [ ]:
# creates matrix where rows are visits containing all the diseases padded with zeros

list_of_visits = []

indexed_data = 'HCUP_D_311_1_to_1_v3.csv'
with open(indexed_data) as input_data:
    cnt_visits = 0
    seen = set()
    index = 0
    for row in input_data:
        visits = row.replace('\n','').split('|')[2].split('#')
        for visit in visits:
            cnt_visits += 1
            v = list(map(int, visit.split(',')[2:]))
            v.sort()
            v.extend([0] * (26 - len(v)))
            v = tuple(v)
            if v not in seen:
                seen.add(v)
                index += 1
                
    list_of_visits = [list(x) for x in seen]
    print(list_of_visits[:5])

In [ ]:
# load medical codes
codes_d = {}
file = 'disease_codes_v2.csv'
with open(file) as input_data:
    index = 0
    for code in input_data:
        code = code.split(',')[1][1:-2]
        codes_d[code] = index
        index +=1 
#print(codes_d)

In [ ]:
# create Frequency Matrix
# creates matrix where rows are visits and columns are diseases codes and values are frequnces for each disease in each visit
freq_matrix = []
for i in range(len(list_of_visits)):
    freq_matrix.append([0] * 1150) # 1023

for i in range(len(list_of_visits)):
    for j in range(len(list_of_visits[0])):
        if list_of_visits[i][j] != 0:
            k = codes_d[str(list_of_visits[i][j])]
            freq_matrix[i][k] += 1

In [ ]:
# runs SVD on freq_matrix
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=50)
visits_emb = svd.fit_transform(freq_matrix)
print(len(visits_emb), len(visits_emb[0]))

In [ ]:
# patients visits
patients_visits = []

indexed_data = 'HCUP_D_311_1_to_1_v3.csv'
with open(indexed_data) as input_data:
    for row in input_data:
        v_codes = []
        visits = row.replace('\n','').split('|')[2].split('#')
        for visit in visits:
            v = list(map(int, visit.split(',')[2:]))
            v.sort()
            v.extend([0] * (26 - len(v)))
            v_codes.append(list_of_visits.index(v))
        patients_visits.append(v_codes)

print(len(patients_visits), len(patients_visits[2]))

In [ ]:
# change a matrix which rows are patients and col contains concatenated feature vectors of that patients visits
rnn_input = []
for i in range(len(patients_visits)):    # 43262
    rnn_input.append([0] * 2500)
    
for i in range(len(patients_visits)):
    for j in range(len(patients_visits[i])):
        rnn_input[i][50 * j: 50 * (j + 1)] = visits_emb[patients_visits[i][j]]

MAKE SURE LENGTH OF ALL ROWS ARE 2500

In [ ]:
# write rnn_input to csv file
import csv
file = 'HCUP_rnn_input.csv'

with open(file, "w", newline="") as new_csv_file:
    writer = csv.writer(new_csv_file)
    for row in rnn_input:
        writer.writerow(row)

In [ ]:
# Take a peak into the new written file for testing purposes
file = 'HCUP_rnn_input.csv'

with open(file) as new_csv_file:
    for line in new_csv_file:
        data = line.split(',')
        print((line))        
        break

**Finally, Machine Learning!**

<hr/>
RNN 
<hr/>

In [ ]:
#Import necessary libraries 
from __future__ import print_function
from keras.callbacks import LambdaCallback
#from keras.models import Sequential
#from tensorflow.keras.models import Sequential
# from keras.layers import Dense, Activation
# from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io
import numpy as np
import pandas as pd

import random
from matplotlib import pyplot as plt
# plt.style.use('dark_background')
from keras.models import Model
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, GlobalMaxPooling1D, SpatialDropout1D, GRU, Bidirectional, Input, TimeDistributed, Reshape

In [ ]:
# load data
rnn_input = []
file = 'HCUP_rnn_input.csv'
with open(file) as input_data:
    for row in input_data:
        row = row.replace('\n','').split(',')
        rnn_input.append(row)

In [ ]:
# change dtype to float
for r in range(0, len(rnn_input)):
    for c in range(0, len(rnn_input[r])):
        rnn_input[r][c] = float(rnn_input[r][c])
print(type(rnn_input[0][0]))

In [ ]:
print(n_pos)

In [ ]:
#print((rnn_input[0]))
y = [1 if i < 135492 else 0 for i in range(135492*2)]
print(len(y))
print(len(rnn_input))

In [ ]:
# 80% training
X = rnn_input
# X = encoded
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size = 0.2, random_state = 42)
print(len(X_train), 'training')
print(len(X_test), 'testing')

Bidirectional GRU

In [ ]:
# Random search drop out rate and number of units/neurons
X_train = np.array(X_train)
X_test = np.array(X_test)
input_shape = (X_train.shape[1], 1)

units = 128


model = Sequential()
model.add(Reshape((50, 50), input_shape=(2500,)))
# 1 sees the past 
# 2 sees the past and the future


dropout = 0.2
print('Number of hidden units: ', units, 'Dropout: ', dropout)
model.add(Bidirectional(GRU(units, input_shape=input_shape))) # 64, $128$, 256, 512
model.add(Dropout(dropout))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)
print('Training...')
history = model.fit(
    np.array(X_train),
    np.array(y_train),
    batch_size = 128, 
    epochs = 11,
    validation_split = 0.1
)

print(model.evaluate(np.array(X_test), y_test), "\n\n")

In [ ]:
plt.clf()
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'g', label='Training loss')
plt.plot(epochs, val_loss, 'y', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
plt.clf()
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
epochs = range(1, len(acc) + 1) #?
plt.plot(epochs, acc, 'g', label='Training acc')
plt.plot(epochs, val_acc, 'y', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

LSTM

In [ ]:
# try 1 way LSTM
# Random search drop out rate and number of units/neurons
X_train = np.array(X_train)
X_test = np.array(X_test)
input_shape = (X_train.shape[1], 1)

units = 64

model = Sequential()
model.add(Reshape((50, 50), input_shape=(2500,)))

# 1 sees the past 
# 2 sees the past and the future

dropout = 1.9
print('Number of hidden units: ', units, 'Dropout: ', dropout)
model.add(LSTM(units, input_shape=input_shape)) # 64, $128$, 256, 512
model.add(Dropout(dropout))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)
print('Training...')
history = model.fit(
    np.array(X_train),
    np.array(y_train),
    batch_size = 128, 
    epochs = 13,
    validation_split = 0.1
)

print(model.evaluate(np.array(X_test), y_test), "\n\n")

We can plot the training and validation accuracy and loss at each epoch by using the history variable returned by the fit function.

In [ ]:
plt.clf()
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'g', label='Training loss')
plt.plot(epochs, val_loss, 'y', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
plt.clf()
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
epochs = range(1, len(acc) + 1) #?
plt.plot(epochs, acc, 'g', label='Training acc')
plt.plot(epochs, val_acc, 'y', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

<hr/>
Classification Using SKlearn - Traditional ML Algorithms
<hr/>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import datasets 
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from time import time

h = .02  # step size in the mesh

names = ["Random Forest", "MLP"]

classifiers = [
    RandomForestClassifier(max_depth=10, n_estimators=100),
    MLPClassifier()
]

print("{0:20}{1:40}\n-----------------------------------------------------".\
      format("Classifier", "Accuracy"))

# iterate over classifiers
for name, clf in zip(names, classifiers):
    
    start_time = time()
    clf.fit(X_train, y_train)
    score = clf.score(X_test, y_test)
    end_time = time()
    print("{0:20}{1:40}{2:40}".format(name, str(score), (end_time - start_time)))